## Dynamic Programming

### 1. Review the course programming code

In [1]:
from collections import defaultdict

In [2]:
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30, 35]
price = defaultdict(int)

for i, p in enumerate(original_price):
    price[i+1] = p

In [3]:
from functools import wraps

In [4]:
def memo(f):
    memo.already_computed = {}
    @wraps(f)
    def _wrap(arg):
        result = None
        if arg in memo.already_computed:
            result = memo.already_computed[arg]
        else:
            result = f(arg)
            memo.already_computed[arg] = result
        return result
    return _wrap

In [5]:
solution = {}

@memo
def r(n):
    max_price, max_split = max(
        [(price[n], 0)] + [(r(i)+r(n-i), i) for i in range(1, n)], key=lambda x: x[0]
         )
    solution[n] = (n-max_split, max_split)
    return max_price

In [6]:
print(r(20))
solution

60


{1: (1, 0),
 2: (2, 0),
 3: (3, 0),
 4: (2, 2),
 5: (3, 2),
 6: (6, 0),
 7: (6, 1),
 8: (6, 2),
 9: (6, 3),
 10: (10, 0),
 11: (11, 0),
 12: (11, 1),
 13: (11, 2),
 14: (11, 3),
 15: (13, 2),
 16: (14, 2),
 17: (11, 6),
 18: (17, 1),
 19: (17, 2),
 20: (17, 3)}

#### 当输入未计算过的长度时抛出异常

In [7]:
def parse_solution(n):
    if n not in solution:
        raise Exception(f'{n} has not been calculated.')
    left, right = solution[n]
    if right == 0: return [left]
    return parse_solution(left) + parse_solution(right)

In [8]:
parse_solution(21)

Exception: 21 has not been calculated.

In [9]:
parse_solution(20)

[11, 6, 3]

### 2.Edit Distance

In [10]:
from functools import lru_cache
solution = {}

In [11]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  
        # string 1 delete tail
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  
        # string 1 add tail of string2
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    solution[(string1, string2)] = operation 
    
    return min_distance

In [12]:
edit_distance('ABCDE', 'DBCCEFG')

4

In [13]:
solution

{('A', 'D'): 'SUB A => D',
 ('A', 'DB'): 'ADD B',
 ('A', 'DBC'): 'ADD C',
 ('A', 'DBCC'): 'ADD C',
 ('A', 'DBCCE'): 'ADD E',
 ('A', 'DBCCEF'): 'ADD F',
 ('A', 'DBCCEFG'): 'ADD G',
 ('AB', 'D'): 'DEL B',
 ('AB', 'DB'): '',
 ('AB', 'DBC'): 'ADD C',
 ('AB', 'DBCC'): 'ADD C',
 ('AB', 'DBCCE'): 'ADD E',
 ('AB', 'DBCCEF'): 'ADD F',
 ('AB', 'DBCCEFG'): 'ADD G',
 ('ABC', 'D'): 'DEL C',
 ('ABC', 'DB'): 'DEL C',
 ('ABC', 'DBC'): '',
 ('ABC', 'DBCC'): 'ADD C',
 ('ABC', 'DBCCE'): 'ADD E',
 ('ABC', 'DBCCEF'): 'ADD F',
 ('ABC', 'DBCCEFG'): 'ADD G',
 ('ABCD', 'D'): '',
 ('ABCD', 'DB'): 'DEL D',
 ('ABCD', 'DBC'): 'DEL D',
 ('ABCD', 'DBCC'): 'SUB D => C',
 ('ABCD', 'DBCCE'): 'ADD E',
 ('ABCD', 'DBCCEF'): 'ADD F',
 ('ABCD', 'DBCCEFG'): 'ADD G',
 ('ABCDE', 'D'): 'DEL E',
 ('ABCDE', 'DB'): 'DEL E',
 ('ABCDE', 'DBC'): 'DEL E',
 ('ABCDE', 'DBCC'): 'DEL E',
 ('ABCDE', 'DBCCE'): '',
 ('ABCDE', 'DBCCEF'): 'ADD F',
 ('ABCDE', 'DBCCEFG'): 'ADD G'}

#### Complete the Edit-Distance Problem Solution, by which we could get the detailed transformer procedure of two string X and Y.

In [14]:
def parse_editting_method(X, Y):
    '''
    -1. 如果X, Y 都为空，那么返回None
    0. 如果X, Y其中有一个为 '': 那么返回一个操作，然后尝试返回parse(X[:-1], Y[:-1])
    1. 如果X，Y找到的答案是 '': 那么就尝试返回parse(X[:-1], Y[:-1])，如果报
    2. 如果X，Y找到的答案是 'SUB ...': 那么就返回替换操作，然后返回parse(X[:-1], Y[:-1])
    3. 如果X，Y找到的答案是 'ADD...' or 'DEL...'，那么就做相应操作，然后返回parse(X+操作, Y)
    '''
    if X == Y: 
        return []
    if X == '' and Y == '': 
        return []
    if X == '': 
        return ["ADD {}".format(Y[-1])] + parse_editting_method(X[:-1], Y[:-1])
    if Y == '': 
        return ["DEL {}".format(X[-1])] + parse_editting_method(X[:-1], Y[:-1])
    if solution[(X, Y)] == '': 
        return parse_editting_method(X[:-1], Y[:-1])
    if solution[(X, Y)].split()[0] == 'SUB': 
        return [solution[(X, Y)]] + parse_editting_method(X[:-1], Y[:-1])
    if solution[(X, Y)].split()[0] == 'ADD': 
        return [solution[(X, Y)]] + parse_editting_method(X+solution[(X, Y)].split()[1], Y)
    if solution[(X, Y)].split()[0] == 'DEL': 
        return [solution[(X, Y)]] + parse_editting_method(X[:-1], Y)

In [15]:
edit_distance('sdfgsdf', 'fgse')

4

In [16]:
parse_editting_method('sdfgsdf', 'fgse')

['DEL f', 'SUB d => e', 'DEL d', 'DEL s']

In [17]:
edit_distance('ABCDE', 'DBCCEFG')

4

In [18]:
parse_editting_method('ABCDE', 'DBCCEFG')

KeyError: ('ABCDEG', 'DBCCEFG')

### 3.Pinyin Auto Correction Problem

In [19]:
chinese_dataset = 'article_9k.txt'
CHINESE_CHARATERS = open(chinese_dataset).read()
CHINESE_CHARATERS[:1000]

'此外自本周6月12日起除小米手机6等15款机型外其余机型已暂停更新发布含开发版体验版内测稳定版暂不受影响以确保工程师可以集中全部精力进行系统优化工作有人猜测这也是将精力主要用到MIUI9的研发之中MIUI8去年5月发布距今已有一年有余也是时候更新换代了当然关于MIUI9的确切信息我们还是等待官方消息\n骁龙835作为唯一通过Windows10桌面平台认证的ARM处理器高通强调不会因为只考虑性能而去屏蔽掉小核心相反他们正联手微软找到一种适合桌面平台的兼顾性能和功耗的完美方案报道称微软已经拿到了一些新的源码以便Windows10更好地理解biglittle架构资料显示骁龙835作为一款集成了CPUGPU基带蓝牙WiFi的SoC比传统的Wintel方案可以节省至少30的PCB空间按计划今年Q4华硕惠普联想将首发骁龙835Win10电脑预计均是二合一形态的产品当然高通骁龙只是个开始未来也许还能见到三星Exynos联发科华为麒麟小米澎湃等进入Windows10桌面平台\n此前的一加3T搭载的是3400mAh电池DashCharge快充规格为5V4A至于电池缩水可能与刘作虎所说一加手机5要做市面最轻薄大屏旗舰的设定有关按照目前掌握的资料一加手机5拥有55寸1080P三星AMOLED显示屏6G8GBRAM64GB128GBROM双1600万摄像头备货量惊喜根据京东泄露的信息一加5起售价是xx99元应该是在279928992999中的某个\n这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车新华社记者张立云摄\n原标题44岁女子跑深圳约会网友被拒暴雨中裸身奔走深圳交警微博称昨日清晨交警发现有一女子赤裸上身行走在南坪快速上期间还起了轻生年头一辅警发现后赶紧为其披上黄衣并一路劝说她那么事发时到底都发生了些什么呢南都记者带您一起还原现场南都记者在龙岗大队坂田中队见到了辅警刘青发现女生的辅警一位外表高大帅气说话略带些腼腆的90后青年刘青介绍6月16日早上7时36分他正在环城南路附近值勤接到中队关于一位女子裸身进入机动车可能有危险的警情随后骑着小铁骑开始沿路寻找大概花了十多分钟在南坪大道坂田出口往龙岗方向的逆行辅道上发现该女子女子身上一丝不挂地逆车流而行时走时停时坐时躺险象环生刘青停好小铁骑和另外一名巡防员追了上去发现女子的情绪很低落话不多刘青尝试和女子交流劝说女子离开可女

In [20]:
from xpinyin import Pinyin

In [21]:
p = Pinyin()

In [22]:
p.get_pinyin('你好', ' ')

'ni hao'

In [23]:
def chinese_to_pinyin(character):
    return p.get_pinyin(character, ' ')

In [24]:
CHINESE_PINYIN_CORPYS = chinese_to_pinyin(CHINESE_CHARATERS)
len(CHINESE_PINYIN_CORPYS)

127826921

In [25]:
import re
from collections import Counter

In [26]:
def tokens(text):
    return re.findall('[a-z]+', text.lower())

In [27]:
tokens('wo bu zhi dao')

['wo', 'bu', 'zhi', 'dao']

In [28]:
PINYIN_COUNT = Counter(tokens(CHINESE_PINYIN_CORPYS))

In [29]:
PINYIN_COUNT.get('yin')

91349

In [30]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

def splits(word):
    "Return a list of all possible (first, rest) pairs that comprise pinyin."
    return [(word[:i], word[i:]) for i in range(len(word)+1)]

def edits0(word):
    "Return all strings that are zero edits away from word (i.e., just word itself)."
    return {word}

def edits1(word):
    "Return all strings that are one edit away from this pinyin."
    pairs = splits(word)
    deletes = [a+b[1:] for (a, b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b)>1]
    replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
    inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "Return all strings that are two edits away from this pinyin."
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

def known(words):
    "Return the pinyin we have noticed."
    return {w for w in words if w in PINYIN_COUNT}

In [31]:
def correct(word):
    "Find the most possible pinyin based on edit distance."
    
    # Prefer edit distance 0, then 1, then 2; otherwise default to word itself.
    
    candidates = (known(edits0(word)) or 
                  known(edits1(word)) or 
                  known(edits2(word)) or 
                  [word])
    return max(candidates, key=PINYIN_COUNT.get)

In [32]:
correct('yign')

'ying'

In [33]:
def correct_sequence_pinyin(text_pingyin):
    return ' '.join(map(correct, text_pingyin.split()))

In [34]:
correct_sequence_pinyin('zhe sih yi ge ce sho')

'zhe shi yi ge ce shi'

### Complete the Pinyin auto-correction problem

In [35]:
CHINESE_PINYIN_CORPYS[:20]

'ci wai zi ben zhou 6'

In [36]:
PINYIN_COUNT.most_common(10)

[('shi', 851326),
 ('de', 809888),
 ('yi', 682635),
 ('ji', 649089),
 ('n', 441587),
 ('guo', 430043),
 ('zhong', 409418),
 ('zhi', 398587),
 ('xin', 359621),
 ('li', 355444)]

In [37]:
PINYIN_tokens = tokens(CHINESE_PINYIN_CORPYS)

#### 删除除了“e”，“o”，“a”的其他不可能为拼音的单个字母

In [38]:
print('n' in PINYIN_tokens)
print('m' in PINYIN_tokens)
print('i' in PINYIN_tokens)

True
True
True


In [39]:
PINYIN_TOKENS = [w for w in PINYIN_tokens if len(w)!=1 or w in ['a', 'o', 'e']]

In [40]:
print('n' in PINYIN_TOKENS)
print('m' in PINYIN_TOKENS)
print('i' in PINYIN_TOKENS)

False
False
False


In [41]:
_2_gram_pinyin_tokens = [' '.join([PINYIN_TOKENS[i], PINYIN_TOKENS[i+1]]) 
                         for i in range(len(PINYIN_TOKENS)-1)]

_2_gram_pinyin_tokens[:20]

['ci wai',
 'wai zi',
 'zi ben',
 'ben zhou',
 'zhou yue',
 'yue ri',
 'ri qi',
 'qi chu',
 'chu xiao',
 'xiao mi',
 'mi shou',
 'shou ji',
 'ji deng',
 'deng kuan',
 'kuan ji',
 'ji xing',
 'xing wai',
 'wai qi',
 'qi yu',
 'yu ji']

In [42]:
_1_gram_COUNTS = Counter(PINYIN_TOKENS)
_1_gram_COUNTS.most_common(10)

[('shi', 851326),
 ('de', 809888),
 ('yi', 682635),
 ('ji', 649089),
 ('guo', 430043),
 ('zhong', 409418),
 ('zhi', 398587),
 ('xin', 359621),
 ('li', 355444),
 ('zai', 334106)]

In [43]:
_2_gram_COUNTS = Counter(_2_gram_pinyin_tokens)
_2_gram_COUNTS.most_common(10)

[('yue ri', 165283),
 ('xin hua', 151826),
 ('hua she', 145946),
 ('zhong guo', 87884),
 ('wai dai', 83330),
 ('nian yue', 78049),
 ('ji zhe', 65393),
 ('er xian', 62431),
 ('dai er', 61784),
 ('zhao pian', 52348)]

In [44]:
_2_gram_COUNTS['wo ai']

223

In [45]:
PINYIN_COUNT['ai']

30281

In [48]:
get_gram_count('woai n', _2_gram_COUNTS)

0

In [49]:
get_gram_count('n', _1_gram_COUNTS)

0

#### 2-gram模型做加一平滑处理，但是不知道这样做对不对

In [47]:
def get_gram_count(word, wc):
    if word in wc: return wc[word]
    else:
        return 0
    
def _2_gram_model(sentence):
    "分子2-gram的count加一，分母next_word的count加句子长度做平滑"
    tokens = sentence.split()
    prob = 1
    for i in range(len(tokens)-1):
        word, next_word = tokens[i], tokens[i+1]
        two_gram_count = get_gram_count(' '.join([word, next_word]), _2_gram_COUNTS)
        one_gram_count = get_gram_count(word, _1_gram_COUNTS)
            
        prob *= (two_gram_count+1)/(one_gram_count + len(tokens))
    return prob

#### 将没有间隔的一句话的拼音split成所有可以的带分隔的句子集，这步就不太会了

In [50]:
import math

In [54]:
sum(_1_gram_COUNTS.values())

30507371

In [59]:
def P(string_list, default=0):
    "返回一列字符串的出现概率（依据词库）"
    probility = 1
    for token in string_list:
        if token not in _1_gram_COUNTS:
            probility *= default
        else:
            probility *= _1_gram_COUNTS.get(token)/sum(_1_gram_COUNTS.values())
    return probility

split_solutions = {}

@lru_cache(maxsize=2**10)
def best_split(string):
    
    notes = [(P([string]), '', string)] + [(best_split(string[:i]) * best_split(string[i:]), string[:i], string[i:]) for i in range(1, len(string))]
    
    prob, left, right = max(notes, key = lambda x: x[0])
    
    split_solutions[string] = (left, right)
    return prob


def parse_split_solution(string):
    left, right = split_solutions[string]
    if not left: 
        return [right]
    return parse_split_solution(left) + parse_split_solution(right)

In [60]:
best_split('zhongguorenmin')

3.9283232283222135e-09

In [61]:
parse_split_solution('zhongguorenmin')

['zhong', 'guo', 'ren', 'min']